In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import os
import sys

src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [3]:
import pandas as pd
import re
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from nbextractor import NBExtractor

In [4]:
print('bla')

bla


In [5]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
def tokenize(s):
    pattern = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    return pattern.sub(r' \1 ', s).split()

In [8]:
X = train.comment_text.values
idx = np.arange(len(X))

In [9]:
train['comment_text'].fillna("unknown", inplace=True)
test['comment_text'].fillna("unknown", inplace=True)

In [10]:
X_train, X_test, idx_train, idx_test = train_test_split(X, idx, test_size=0.2)

In [11]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                        min_df=3, max_df=0.9, strip_accents='unicode',
                        use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')

In [12]:
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [14]:
probs = np.zeros(shape=(len(X_test), 6))
for i, col in enumerate(train.columns[2:]):
    print("Training ", col)
    y_train = train.loc[idx_train, col].values
    y_test = train.loc[idx_test, col].values
    nbext = NBExtractor(alpha=1).fit(X_train_vec, y_train)
    X_train_vec_nb = nbext.transform(X_train_vec)
    logc = LogisticRegression(n_jobs=7, verbose=True)
    logc.fit(X_train_vec_nb, y_train)
    y_pred = logc.predict_proba(nbext.transform(X_test_vec))[:,1]
    probs[:, i] = y_pred
    auc = roc_auc_score(y_test, y_pred)
    lg_loss = log_loss(y_test, y_pred)
    print("{} auc: {}".format(col, auc))
    print("{} log loss: {}".format(col, lg_loss))

Training  toxic


/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 7.
  " = {}.".format(self.n_jobs))


[LibLinear]toxic auc: 0.9777447068399512
toxic log loss: 0.10670662259047393
Training  severe_toxic
[LibLinear]severe_toxic auc: 0.9846121915286362
severe_toxic log loss: 0.026303645604310293
Training  obscene
[LibLinear]obscene auc: 0.9909084704809531
obscene log loss: 0.05665615431315777
Training  threat
[LibLinear]threat auc: 0.9819170271658959
threat log loss: 0.010954123575106459
Training  insult
[LibLinear]insult auc: 0.9800228668646502
insult log loss: 0.07951599533179825
Training  identity_hate
[LibLinear]identity_hate auc: 0.9714843970653424
identity_hate log loss: 0.026580362309553428


In [ ]:
pipeline = Pipeline([
    ('naive_bayes_extractor', NBExtractor()),
    ('svc', SVC(verbose=True))
])

In [20]:
pipeline.fit(X_train_vec, train.loc[idx_train, 'toxic'].values)

Pipeline(memory=None,
     steps=[('naive_bayes_extractor', <nbextractor.NBExtractor object at 0x7f8fd8a36518>), ('logistic_regression', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [21]:
pipeline.predict(X_test_vec)

array([0, 0, 0, ..., 0, 0, 0])

In [22]:
pipeline.predict_proba(X_test_vec)

AttributeError: predict_proba is not available when  probability=False

In [ ]:
preds = pd.DataFrame()
preds['id'] = test.id

In [ ]:
X_train_vec = tfidf.fit_transform(X)
X_test = test.comment_text.values
X_test_vec = tfidf.transform(X_test)

In [ ]:
for col in train.columns[2:]:
    print("Training ", col)
    y_train = train.loc[:, col].values
    nblog = NbLogisticClassifier(n_jobs=-1)
    nblog.fit(X_train_vec, y_train)
    preds[col] = nblog.predict_proba(X_test_vec)[:, 1]

In [ ]:
preds.to_csv('../submissions/nbsvm-baseline.csv', index=False)

In [ ]:
sample = pd.read_csv('../data/sample_submission.csv').head()

In [ ]:
pd.read_csv('../submissions/nbsvm-baseline.csv').head()